In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
from decimal import Decimal
from datetime import datetime

%matplotlib inline

import findspark
findspark.init()

from pyspark.sql import *
from pyspark.sql.functions import  isnan, when, count, col,isnull

%matplotlib inline

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [2]:
spark

In [3]:
sns.set_context('talk')
%matplotlib inline

sns.set_style("darkgrid", {"axes.facecolor": ".9"})
sns.set_context('notebook')

In [4]:
data_folder = './data/'

In [6]:
data  = spark.read.csv(data_folder+'en.openfoodfacts.org.products.csv',header = True, sep="\t")

In [12]:
data.columns

['code',
 'url',
 'creator',
 'created_t',
 'created_datetime',
 'last_modified_t',
 'last_modified_datetime',
 'product_name',
 'generic_name',
 'quantity',
 'packaging',
 'packaging_tags',
 'brands',
 'brands_tags',
 'categories',
 'categories_tags',
 'categories_en',
 'origins',
 'origins_tags',
 'manufacturing_places',
 'manufacturing_places_tags',
 'labels',
 'labels_tags',
 'labels_en',
 'emb_codes',
 'emb_codes_tags',
 'first_packaging_code_geo',
 'cities',
 'cities_tags',
 'purchase_places',
 'stores',
 'countries',
 'countries_tags',
 'countries_en',
 'ingredients_text',
 'allergens',
 'allergens_en',
 'traces',
 'traces_tags',
 'traces_en',
 'serving_size',
 'serving_quantity',
 'no_nutriments',
 'additives_n',
 'additives',
 'additives_tags',
 'additives_en',
 'ingredients_from_palm_oil_n',
 'ingredients_from_palm_oil',
 'ingredients_from_palm_oil_tags',
 'ingredients_that_may_be_from_palm_oil_n',
 'ingredients_that_may_be_from_palm_oil',
 'ingredients_that_may_be_from_palm_

In [8]:
selected = data.select(['code','countries','categories','origins','nutrition-score-fr_100g','packaging','-palmitic-acid_100g'])

In [9]:
selected.count()

692892

In [96]:
ciao = data.select([(count(when(isnull(c), c))).alias(c) for c in data.columns])
ciaone = ciao.toPandas()

In [102]:
pd.options.display.max_columns = 100000000
ciaone[ciaone<300000].dropna(axis=1)

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,brands,brands_tags,countries,countries_tags,countries_en,ingredients_text,serving_quantity,additives_n,additives,ingredients_from_palm_oil_n,ingredients_that_may_be_from_palm_oil_n,states,states_tags,states_en,energy_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,salt_100g,sodium_100g
0,34,34,3,4,10,0,0,26698,224276,224316,548,548,548,294125,18772,294159,294172,294159,294159,68,68,68,100119,105591,131379,105837,116107,101555,121352,121388


In [40]:
palm_oil = data.select(['countries','ingredients_from_palm_oil_n','ingredients_that_may_be_from_palm_oil_n'])

In [53]:
palm_oil.count()

692892

In [42]:
palm_oil_noNAN = palm_oil.dropna()

In [43]:
palm_oil_noNAN.count()

392887

In [51]:
palm_oil_noNAN.filter('ingredients_from_palm_oil_n != 0 ').count()

11960

In [54]:
palm_oil_flat = palm_oil.rdd.flatMap(lambda r: [(x, r['ingredients_from_palm_oil_n'], r['ingredients_that_may_be_from_palm_oil_n']) 
                                                for x in r['Countries']])

In [58]:
palm_oil_flat.take(3)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 24.0 failed 1 times, most recent failure: Lost task 0.0 in stage 24.0 (TID 128, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "D:\Documents\etudes\epfl\cours\Applied_Data_Analysis\spark\spark-2.3.2-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\sql\types.py", line 1543, in __getitem__
    idx = self.__fields__.index(item)
ValueError: 'Countries' is not in list

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\Documents\etudes\epfl\cours\Applied_Data_Analysis\spark\spark-2.3.2-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 253, in main
  File "D:\Documents\etudes\epfl\cours\Applied_Data_Analysis\spark\spark-2.3.2-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker